# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
# Bayesian Parameter Sampling
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.core import Environment, ScriptRunConfig
import os
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice




## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
experiment_name = 'customized-rain'

experiment=Experiment(ws, experiment_name)
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

# Create a Cluster

In [ ]:

#cluster_name = "GIVE_A_CLUSTER_NAME"
cluster_name = "capstone"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

try:
    cpu_cluster=ComputeTarget(workspace=ws,name=cluster_name)
    print('Cluster already exists!')
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",max_nodes=4)
    cpu_cluster = ComputeTarget(ws,cluster_name,compute_config)
cpu_cluster.wait_for_completion(show_output=True)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
# I am going to put Bayesian termination policy
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.15, slack_amount=None, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
# this is easy, from LR,C strength and max iter
param_sampling = BayesianParameterSampling({
    '--C':choice(0.01,0.1,1.0,10.,100.),
    "--max_iter":choice(100,500,1000)
})

#TODO: Create your estimator and hyperdrive config
# Setup environment for your training run
estimator = Environment.from_conda_specification(name='sklearn-env', 
                                                   file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                            script='train_raintomorrow.py',
                            compute_target=cpu_cluster,
                            environment=estimator)### YOUR CODE HERE ###

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                    hyperparameter_sampling=param_sampling,
                    policy=early_termination_policy,
                    primary_metric_name='Accuracy',
                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                    max_total_runs=4,
                    max_concurrent_runs=4)### YOUR CODE HERE ###



In [ ]:
#TODO: Submit your experiment
run_hyperdrive=experiment.submit(config=hyperdrive_config)
run_hyperdrive.wait_for_completion(show_output=True)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(run_hyperdrive).show()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = run_hyperdrive.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Best run metrics:', best_run_metrics)
print('\n parameter values:', parameter_values)


In [ ]:
#TODO: Save the best model
saved_model=best_run.register_model(model_name='best-hyperdrive',model_path='./')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# need to register both customized and AutoML best models

# from: "https://learn.microsoft.com/en-us/azure/machine-learning/v1/how-to-deploy-and-where?tabs=python"
# Register model
model = Model.register(ws, model_name="customized_model", model_path="./best-hyperdrive")



service_name = 'bestmodel-service'

inference_config = InferenceConfig(entry_script='score.py', environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=8000)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import requests
import json

uri = service.scoring_uri
requests.get("http://localhost:8000")
headers = {"Content-Type": "application/json"}
data = {
    "query": "What color is the fox",
    "context": "The quick brown fox jumped over the lazy dog.",
}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())
service.delete()

In [ ]:
cpu_cluster.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

